In [ ]:
!pip install googletrans==3.1.0a0

In [ ]:
import pandas as pd
from googletrans import Translator

In [ ]:
df = pd.read_csv('raw_data/Clustering_ProductNodeCategory.csv')

In [ ]:
df

,ASIN,REGION,MARKETPLACE_NAME,COUNTRY,BRAND,MIN_CATEGORY_LEAF_NAME
0,B01HZDQ5Q2,North America,amazon.us,United States,Espring,Muffin & Cupcake Pans
1,B0BS72PJ65,North America,amazon.us,United States,Espring,Bungee Cords
2,B018I2NCBA,North America,amazon.us,United States,Espring,Brushes
3,B07SYKTLG9,North America,amazon.us,United States,Espring,Miniatures
4,B07X55ZZKR,North America,amazon.us,United States,Espring,NaN
...,...,...,...,...,...,...
8105,B092D5D8T3,Europe,amazon.tr,Turkey,Tritart,Origami Kağıtları
8106,B01H723G6S,Europe,amazon.nl,Netherlands,Espring,Badkussens
8107,B07DPRTTQ9,Europe,amazon.tr,Turkey,H&S,Pipetler
8108,B0829BCTTX,Europe,amazon.tr,Turkey,Tritart,Origami Kağıtları


In [ ]:
df['MIN_CATEGORY_LEAF_NAME'].unique()

array(['Muffin & Cupcake Pans', 'Bungee Cords', 'Brushes', ...,
       'Hediye Torbaları', 'Simler', 'Badkussens'], dtype=object)

In [ ]:
len(df['MIN_CATEGORY_LEAF_NAME'].unique())

2896

---
Detect node category language and translate to english if necessary

In [ ]:
translator = Translator()

lang_detect = lambda x : translator.detect(x).lang
to_english = lambda x : translator.translate(x, dest='en').text

In [ ]:
#takes a while to execute. Haven't managed to optimize this line unfortunately.

df['detected_lang'] = df['MIN_CATEGORY_LEAF_NAME'].apply(lang_detect)

In [ ]:
non_english_df = df[df['detected_lang'] != "en"]
english_df = df[df['detected_lang'] == "en"]
english_df['detected_lang'] = "en"


<ipython-input-45-b57a7c487465>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  english_df['detected_lang'] = "en"
<ipython-input-45-b57a7c487465>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_english_df['MIN_CATEGORY_LEAF_NAME_ENG'] = non_english_df['MIN_CATEGORY_LEAF_NAME'].apply(to_english)


In [ ]:
#takes a while to execute too.

non_english_df['MIN_CATEGORY_LEAF_NAME_ENG'] = non_english_df['MIN_CATEGORY_LEAF_NAME'].apply(to_english)
non_english_df2 = non_english_df.drop(columns=['MIN_CATEGORY_LEAF_NAME']).rename(columns={"MIN_CATEGORY_LEAF_NAME_ENG":"MIN_CATEGORY_LEAF_NAME"})

In [ ]:
df2 = pd.concat([non_english_df2, english_df]).sort_index()

In [ ]:
df2['COUNTRY'].unique()

array(['United States', 'United Kingdom', 'Canada', 'Belgium', 'Germany',
       'Spain', 'Mexico', 'France', 'Italy', 'Poland', 'Sweden', 'Turkey',
       'Netherlands'], dtype=object)

In [ ]:
#to replace COUNTRY by MARKETPLACE code, to match other data sources in the future.
countries = ['United States', 'United Kingdom', 'Canada', 'Belgium', 'Germany',
       'Spain', 'Mexico', 'France', 'Italy', 'Poland', 'Sweden', 'Turkey', 'Netherlands']
codes = ['US', 'UK', 'CA','BE','DE', 'ES', 'MX','FR','IT','PL','SE','TR','NL']

map_marketplace = dict(zip(countries, codes))

df2['COUNTRY'] = df2['COUNTRY'].map(map_marketplace)

In [ ]:
df2 = df2.rename(columns={'COUNTRY':"MARKETPLACE", "BRAND":"BRAND_NAME"})

In [ ]:
#group by asin/market/brand, count unique min_category values and create a string joining all values
final_node_categories = df2.groupby(['ASIN','MARKETPLACE','BRAND_NAME']).agg({'MIN_CATEGORY_LEAF_NAME':['nunique',lambda x : " ".join(x)]}).reset_index()

final_node_categories.columns = ['ASIN','MARKETPLACE','BRAND_NAME','NUM_CAT','MIN_CATEGORY_LEAF_NAME']
final_node_categories = final_node_categories[['ASIN','MARKETPLACE','BRAND_NAME','MIN_CATEGORY_LEAF_NAME']]
final_node_categories = final_node_categories[final_node_categories['MIN_CATEGORY_LEAF_NAME'] != "in"]

In [ ]:
final_node_categories

,ASIN,MARKETPLACE,BRAND_NAME,MIN_CATEGORY_LEAF_NAME
0,B00414KXNU,BE,Tritart,Paint brushes
1,B00414KXNU,DE,Tritart,"Pinsel-Sets Customer favorites: kitchen, house..."
2,B00414KXNU,ES,Tritart,Brush and brush sets
4,B00414KXNU,IT,Tritart,The products most loved by our customers: Furn...
5,B00414KXNU,NL,Tritart,Penseelsets
...,...,...,...,...
7730,B0CGJ4H9GD,IT,H&S,Decorative money boxes
7731,B0CGJ5XXG3,ES,H&S,Home and work
7733,B0CGJLPVZ7,UK,Tritart,Easels
7734,B0CGJM2VV8,ES,Tritart,Easels


In [ ]:
final_node_categories.to_csv('preprocessed_data/Product_NodeCategories.csv',index=False)